<a href="https://colab.research.google.com/github/IzzulGod/GPT2-Small-Indonesian/blob/main/evol_fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft accelerate
!pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("cahya/gpt2-small-indonesian-522M")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "cahya/gpt2-small-indonesian-522M",
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/evol-instruct-indonesian")

def convert_to_single_turn(examples):
    results = []
    for conversations_list in examples["conversations"]:
        for i in range(len(conversations_list) - 1):
            m1 = conversations_list[i]
            m2 = conversations_list[i + 1]

            if m1["from"] == "human" and m2["from"] == "gpt":
                user_msg = m1["value"].strip()
                bot_msg = m2["value"].strip()
                combined = f"User: {user_msg}\nAI: {bot_msg} <|endoftext|>"
                results.append({"text": combined})
    return {"text": results}

dataset = dataset.map(
    convert_to_single_turn,
    batched=True,
    remove_columns=["conversations", "id"]
)

for i in range(3):
    print(f"\n=== Sampel {i+1} ===")
    print(dataset["train"][i]["text"])

print(f"\nJumlah sampel single-turn: {len(dataset['train'])}")

def tokenize(batch):
    tokens = tokenizer(
        [example["text"] for example in batch["text"]],  # Extract the string from the dictionary
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=dataset["train"].column_names
)

sample_texts = [dataset["train"][i]["text"] for i in range(3)]

tokenized_batch = tokenizer([sample["text"] for sample in sample_texts], padding="max_length", truncation=True, max_length=512) # Extract the string from the dictionary

for i in range(3):
    print(f"\n🔹 Sampel {i+1}")
    print("="*40)
    print("📄 Teks Asli:\n")
    print(sample_texts[i])
    print("\n🔢 Token IDs:\n")
    print(tokenized_batch["input_ids"][i])
    print("\n🔤 Decoded dari Token:\n")
    print(tokenizer.decode(tokenized_batch["input_ids"][i]))
    print("="*40)

Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/59022 [00:00<?, ? examples/s]


=== Sampel 1 ===
{'text': 'User: Apa kode R untuk membuat dataframe dengan tiga kolom bernama "name", "age", dan "salary"? Kolom "name" harus memiliki nilai "John", "Mary", dan "Peter". Kolom "age" harus memiliki nilai 25, 30, dan 35. Kolom "salary" harus memiliki nilai 50000, 60000, dan 70000. Mohon berikan solusi [kode R] untuk membuat dataframe ini.\nAI: Untuk membuat dataframe dengan tiga kolom bernama "name", "age", dan "salary", serta dengan nilai-nilai yang telah disebutkan, dapat dilakukan dengan menggunakan kode R sebagai berikut:\n\n```\ndf <- data.frame(name=c("John", "Mary", "Peter"), age=c(25, 30, 35), salary=c(50000, 60000, 70000))\n```\n\nPenjelasan kode diatas:\n\n- `data.frame()`: membuat sebuah dataframe baru dengan argumen yang diinputkan di dalamnya.\n- `name=c("John", "Mary", "Peter")`: nilai parameter `name` yang terdiri dari vektor karakter "John", "Mary", dan "Peter".\n- `age=c(25, 30, 35)`: nilai parameter `age` yang terdiri dari vektor numerik 25, 30, dan 35.

Map:   0%|          | 0/59022 [00:00<?, ? examples/s]


🔹 Sampel 1
📄 Teks Asli:

{'text': 'User: Apa kode R untuk membuat dataframe dengan tiga kolom bernama "name", "age", dan "salary"? Kolom "name" harus memiliki nilai "John", "Mary", dan "Peter". Kolom "age" harus memiliki nilai 25, 30, dan 35. Kolom "salary" harus memiliki nilai 50000, 60000, dan 70000. Mohon berikan solusi [kode R] untuk membuat dataframe ini.\nAI: Untuk membuat dataframe dengan tiga kolom bernama "name", "age", dan "salary", serta dengan nilai-nilai yang telah disebutkan, dapat dilakukan dengan menggunakan kode R sebagai berikut:\n\n```\ndf <- data.frame(name=c("John", "Mary", "Peter"), age=c(25, 30, 35), salary=c(50000, 60000, 70000))\n```\n\nPenjelasan kode diatas:\n\n- `data.frame()`: membuat sebuah dataframe baru dengan argumen yang diinputkan di dalamnya.\n- `name=c("John", "Mary", "Peter")`: nilai parameter `name` yang terdiri dari vektor karakter "John", "Mary", dan "Peter".\n- `age=c(25, 30, 35)`: nilai parameter `age` yang terdiri dari vektor numerik 25, 30,

In [6]:
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 9,437,184 || all params: 133,876,992 || trainable%: 7.0491


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [7]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-chatbot-lora",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=200,
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    fp16=True,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    report_to="none"
)

train_dataset = tokenized_dataset["train"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,3.533500
400,2.964200
600,2.847200
800,2.772600
1000,2.717300
1200,2.671700
1400,2.651500
1600,2.623400
1800,2.586100
2000,2.551900


TrainOutput(global_step=5535, training_loss=2.5733587828431994, metrics={'train_runtime': 12603.3708, 'train_samples_per_second': 14.049, 'train_steps_per_second': 0.439, 'total_flos': 5.139926052293837e+16, 'train_loss': 2.5733587828431994, 'epoch': 3.0})

In [8]:
model.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")


('/content/drive/MyDrive/gpt2-lora-adapter/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/vocab.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/merges.txt',
 '/content/drive/MyDrive/gpt2-lora-adapter/added_tokens.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/tokenizer.json')

In [9]:
from peft import PeftModel

if isinstance(model, PeftModel):

    merged_model = model.merge_and_unload()

    merged_model.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
    tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
else:
    print("tidak ada adapter untuk digabungkan")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/gpt2-lora-merged"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cpu")

prompt = "User: Siapa presiden pertama Indonesia?\nAI:"
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
